# Regressão Linear Simples

Neste notebook será utilizada uma base de dados com informações sobre transações imobiliárias para se realizar a tarefa de predição usando regressão linear simples (somente uma entrada).
As seguintes ações foram realizadas:
* Usar funções do Pandas (API para análise e estruturação de dados)
* Desenvolver uma função para computar os coeficientes de uma regressão linear simples usando a solução de forma fechada
* Desenvolver uma função para realizar predições da saída desejada, dado o atributo de entrada
* Utilizar a função de regressão para realizar predições de valor de venda de casas dado a metragem da mesma
* Comparar dois diferentes modelos para predição dos valores de casas

## Importação das bibliotecas

Importação de bibliotecas necessárias para a execução dos comandos:
* Pandas: para manipulação dos dados
* Numpy: para compatibilizar o uso de dados por algumas funções
* Datetime: para formatar um dos atributos da base de dados
* Sklearn: para uso comparativo de funções de regressão linear

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.linear_model import LinearRegression

### Leitura dos dados

O dataset contém dados de transações imobiliárias de King County, região aonde está a cidade de Seattle, USA.

In [3]:
sales = pd.read_csv("kc_house_data.csv")

In [4]:
sales

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [5]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
sales = pd.read_csv("kc_house_data.csv", dtype=dtype_dict)
train_data = pd.read_csv("kc_house_train_data.csv", dtype=dtype_dict)
test_data = pd.read_csv("kc_house_test_data.csv", dtype=dtype_dict)

sales["date"] = pd.to_datetime(sales["date"])
train_data["date"] = pd.to_datetime(train_data["date"])
test_data["date"] = pd.to_datetime(test_data["date"])


### Teste de uso de funções

Teste de uso de algumas funções que podem ser realizadas sobre Dataframes do Pandas. Neste caso em específico estamos testando a função para calcular a média de valores numéricas aplicada a um objeto do tipo ***pandas.core.series.Series***

In [6]:
prices = sales['price'] # extract the price column of the sales SFrame -- this is now an SArray
avg = prices.mean() # if you just want the average, the .mean() function
print("Preço médio: " + str(avg))

Preço médio: 540088.1417665294


## Construindo uma função genérica de regressão linear simples

Utilizando funções da biblioteca Numpy, podemos utilizar a solução de forma fechada para computar o slope e o intercept para uma regressão linear simples a partir de observações extraídas dos arrays input_feature e output.

In [7]:
def simple_linear_regression(input_feature, output):
    # compute the sum of input_feature and output
    input_feature_sum = input_feature.sum() #10
    output_sum = output.sum() #15
    
    #compute the number of datapoints
    N = input_feature.size #5
    
    # compute the product of the output and the input_feature and its sum
    product_input_output = (input_feature*output).sum() #40
    
    # compute the squared value of the input_feature and its sum
    squared_input_feature = (input_feature**2).sum() #30
    
    # use the formula for the slope
    numerator = product_input_output - (input_feature_sum * output_sum)/N #40-(150/5) = 2,5
    denominator = squared_input_feature - (input_feature_sum * input_feature_sum)/N #30-(150/5)
    slope = numerator/denominator
    
    # use the formula for the intercept
    intercept = output_sum/N - slope * input_feature_sum/N
    
    
    return (intercept, slope)

Podemos testar se a função acima está funcionando passando para ela algo sobre o qual já sabemos a resposta. Em particular, podemos gerar um feature (atributo) e então colocar o output (saída) exatamente em uma linha: output = 1 + 1*input_feature. Desta forma sabemos que tanto o nosso slope quanto o intercept deve conter o valor 1.

In [8]:
test_feature = np.array([range(5)])
print(test_feature)
test_output = (1 + 1*test_feature)
print(test_output)
(test_intercept, test_slope) =  simple_linear_regression(test_feature, test_output)
print("Intercept: " + str(round(test_intercept)))
print("Slope: " + str(round(test_slope)))

[[0 1 2 3 4]]
[[1 2 3 4 5]]
Intercept: 1
Slope: 1


Agora que sabemos que a função está funcionando, vamos construir um modelo de regressão para realizar a predição baseada no atributo que contém a metragem da casa (sqft_living). É importante lembrar que o modelo deve ser treinando utilizando a base train_data!

In [9]:
sqft_intercept, sqft_slope = simple_linear_regression(train_data['sqft_living'], train_data['price'])

print("Intercept: " + str(sqft_intercept))
print("Slope: " + str(sqft_slope))

Intercept: -47116.07907289418
Slope: 281.9588396303426


O código abaixo faz a mesma coisa que o código acima. Contudo, estamos fazendo uso de funções pré-existentes da API do Scikit-learn para criar um modelo de regressão linear simples. Observe que tanto o intercept quanto o slope contém os mesmos valores encontrados por nossa função.

In [10]:
# Create linear regression object
regr = LinearRegression()
regr.fit(train_data["sqft_living"].values.reshape(train_data["sqft_living"].size, 1), train_data["price"])
print("Intercept: ", regr.intercept_)
print("Slope: ", regr.coef_[0])

Intercept:  -47116.07907289383
Slope:  281.9588396303424


## Realizando a predição de valores

Agora que temos os parâmetros do modelo: intercept e slope, podemos realizar as predições. Usando numpy.array fica fácil multiplicá-lo por uma constante e adicionar a ele um outro valor constante.

In [11]:
def get_regression_predictions(input_feature, intercept, slope):
    # calculate the predicted values:
    predicted_values = intercept + slope*input_feature
    
    return predicted_values

Agora que podemos realizar predição de dados com o slope e o intercept, vamos fazer uma predição. Vamos tentar encontrar o valor estimado do preço de uma casa com 2.650 m2 a partir da metragem da mesma, de acordo com o modelo de regressão estimado acima.

In [12]:
my_house_sqft = 2640.00
estimated_price = get_regression_predictions(my_house_sqft, sqft_intercept, sqft_slope)
print("O preço destimado de uma casa com %d metros quadrados é $%.2f" % (my_house_sqft, estimated_price))

O preço destimado de uma casa com 2640 metros quadrados é $697255.26


## Método de mínimos quadrados (RSS)

Agora que temos um modelo que pode realizar predições, vamos avaliar o modelo usando o método de mínimos quadrados (RSS). Lembre-se que RSS é a soma dos quadrados dos erros residuais e os residuais são os valores de diferença entre a saída prevista e a saída real.

In [13]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    # First get the predictions
    predictions  = get_regression_predictions(input_feature, intercept, slope)

    # then compute the residuals (since we are squaring it doesn't matter which order you subtract)
    predicted_values = output - (intercept + slope*input_feature)

    # square the residuals and add them up
    RSS = float((predicted_values**2).sum())

    return(RSS)

Vamos testar nossa função get_residual_sum_of_squares aplicando-a ao modelo de teste aonde os dados estão exatamente em uma linha. Desde que eles estão exatamente em uma linha, a soma residual dos quadrados deve ser zero !

In [14]:
print(get_residual_sum_of_squares(test_feature, test_output, test_intercept, test_slope)) # should be 0.0

0.0


Agora vamos usar a função para calcular o RSS sobre os dados de treinamento do modelo calculado acima.

In [15]:
rss_prices_on_sqft = get_residual_sum_of_squares(train_data['sqft_living'], train_data['price'], sqft_intercept, sqft_slope)
print('O erro RSS para predição de preços baseados na metragem é: ' + str(rss_prices_on_sqft))

O erro RSS para predição de preços baseados na metragem é: 1201918354177283.0


## Prevendo a metragem dado o preço

E se quisermos prever a metragem dado o preço ? Desde que nós tenhamos uma equação y = a + b\*x nós podemos resolver a função para x. Assim se tivermos o intercept (a) e o slope (b) e o preço (y) nós podemos computar a metragem estimada.

In [16]:
def inverse_regression_predictions(output, intercept, slope):
    # solve output = intercept + slope*input_feature for input_feature. Use this equation to compute the inverse predictions:
    estimated_feature = (output-intercept)/float(slope)

    return estimated_feature

Agora que temos uma função para computar a metragem dado o preço a partir do nosso modelo de regressão simples, vamos ver o quanto podemos esperar de uma casa que custa $800.000.

In [17]:
my_house_price = 800000
estimated_squarefeet = inverse_regression_predictions(my_house_price, sqft_intercept, sqft_slope)
print("A metragem estimada para uma casa que vale $%.2f é de %dm2" % (my_house_price, estimated_squarefeet))

A metragem estimada para uma casa que vale $800000.00 é de 3004m2


## Novo modelo: estimar preços a partir da quantidade de quartos

Nós criamos um modelo para prever preços de casas usando a metragem, mas há vários outros atributos no DataFrame.
Vamos usar a nossa função de regresão linear simples para estimar os parâmetros de regressão a partir da predição de preços baseados no número de quartos. Vamos utilizar os dados de treinamento.

In [18]:
# Estimate the slope and intercept for predicting 'price' based on 'bedrooms'
bedrooms_intercept, bedrooms_slope = simple_linear_regression(train_data['bedrooms'], train_data['price'])

print("Intercept: " + str(bedrooms_intercept))
print("Slope: " + str(bedrooms_slope))

Intercept: 109473.1776229596
Slope: 127588.95293398784


Atividade em sala: Novo Modelo, estimando preço baseado na quantidade de banheiros

In [19]:
bathrooms_intercept, bathrooms_slope = simple_linear_regression(train_data['bathrooms'], train_data['price'])

print("Intercept: " + str(bathrooms_intercept))
print("Slope: " + str(bathrooms_slope))

Intercept: 9151.221709646634
Slope: 250687.13614554782


Desafio em sala de aula: Tentando fazer regressão linear múltipla.

In [20]:
bathrooms_intercept, bathrooms_slope = simple_linear_regression(train_data['bathrooms'], train_data['price'])

print("Intercept: " + str(bathrooms_intercept))
print("Slope: " + str(bathrooms_slope))

Intercept: 9151.221709646634
Slope: 250687.13614554782


## Testando o algoritmo de regressão linear

Agora temos dois modelos para predição do preço de uma casa. Como saber qual é o melhor ? Vamos calcular o RSS sobre os dados de test (lembre-se que estes dados não foram envolvidos no modelo de aprendizado). Compute o RSS a partir da predição de preços usando o número de quartos e da predição de preços usando a metragem.

In [21]:
# Compute RSS when using bedrooms on TEST data:
rss_prices_on_bedrooms = get_residual_sum_of_squares(test_data['bedrooms'], test_data['price'], bedrooms_intercept, bedrooms_slope)
print('O erro RSS para predição de preços baseados na quantidade de quartos é: ' + str(rss_prices_on_bedrooms))

O erro RSS para predição de preços baseados na quantidade de quartos é: 493364585960300.9


In [22]:
# Compute RSS when using squarefeet on TEST data:
rss_prices_on_sqft = get_residual_sum_of_squares(test_data['sqft_living'], test_data['price'], sqft_intercept, sqft_slope)
print('O erro RSS para predição de preços baseados na metragem é: ' + str(rss_prices_on_sqft))

O erro RSS para predição de preços baseados na metragem é: 275402933617812.12


## Tentando Fazer Regressão Linear Múltipla (25/10/2023)

Na regressão linear múltipla utilizamos vários atributos (features) para treinar e testar nosso modelo, precisamos primeiramente selecionar e separar esses atributos.

In [27]:
train_data = pd.read_csv("kc_house_train_data.csv", dtype=dtype_dict)
test_data = pd.read_csv("kc_house_test_data.csv", dtype=dtype_dict)
train_multi = pd.read_csv()

In [29]:
regr=LinearRegression()
regr.fit(train_data["sqft_living"].values.reshape(train_data["sqft_living"].size, 1), train_data["price"])
print("Intercept: ", regr.intercept_)
print("Slope 1: ", regr.coef_[0])
print("Slope 2: ", regr.coef_[1])
print("Slope 3: ", regr.coef_[2])

KeyError: ('sqft_living', 'bathrooms', 'bedrooms')